In [1]:
from google.colab import drive
drive.mount('/content/drive')                                                                                                                                                                                                                                          

Mounted at /content/drive


In [2]:
#!ls '/content/drive/My Drive/Projects/RL/gym-arobo'
%cd '/content/drive/My Drive/Projects/RL/gym-arobo'
#!ls
!pip install -e .

/content/drive/My Drive/Projects/RL/gym-arobo
Obtaining file:///content/drive/My%20Drive/Projects/RL/gym-arobo
     |████████████████████████████████| 11.8MB 239kB/s 
  Running setup.py develop for gym-arobo


In [3]:
!pip install statistics 
!pip install tflearn

import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((700,710))

  Created wheel for statistics: filename=statistics-1.0.3.5-cp37-none-any.whl size=7454 sha256=7f321d07aff37e542b6f0a1e366a648525bba8ee0390d92da397586f0af815c3
  Stored in directory: /root/.cache/pip/wheels/75/55/90/73aa7662bfb4565b567618547a275f01372a678ca92ecd64f3
Successfully built statistics
     |████████████████████████████████| 112kB 18.8MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp37-none-any.whl size=127301 sha256=8e08be93e72f989f9a906566549c99bd5dfbb3ba1a8e31b8a6a3aef0bfcae6df
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn
pygame 2.0.1 (SDL 2.0.14, Python 3.7.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


<Surface(700x710x32 SW)>

In [4]:
import gym
import gym_arobo
import random 
import numpy as np
from statistics import median, mean
from collections import Counter
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.models import load_model

import tflearn
import cv2
import tensorflow as tf
from collections import deque
from tflearn.layers.estimator import regression
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d
from tflearn.optimizers import RMSProp

path = '/content/drive/My Drive/Projects/RL/Saved Models/'

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
game = "ARobo-v0"
env = gym.make(game)
env.reset()

def get_epsilon(epsilon, gen):
    epsilon = epsilon
    epsilon -= gen * 0.001
    print(epsilon)
    return epsilon

LR = 1e-3
num_games = 50  
goal_steps = 1000
possible_actions = env.action_space.n
MODEL_NAME = path + 'Model_{}'
gamma = 0.99
epsilon = 1
generations = 50	
#height = 84
#width = 84

In [6]:
def bot_velocity(target_x,target_y,x,y):

	x_reach = target_x + 5
	y_reach = target_y + 5
	dir_x = 1 #directions to travel for bot
	dir_y = 1
	dist_x = abs(x_reach - x)
	dist_y = abs(y_reach - y)
	
	if x_reach-x >= 0:
		dir_x = 1
	else:
		dir_x = -1
	
	if y_reach-y >= 0:
		dir_y = 1
	else:
		dir_y = -1 
	
	"""
	ACTIONS:
	
	Nothing ----> 0
	Left -------> 1
	Right ------> 2
	Up ---------> 3
	Down -------> 4
	"""
	
	if dist_x==0:
		if dir_y==1:
			action = 4
		else:
			action = 3
	elif dist_y==0:
		if dir_x==1:
			action = 2
		else:
			action = 1
	else:
		if dist_x>=dist_y:
			if dir_x==1:
				action = 2
			else:
				action = 1
		else:
			if dir_y==1:
				action = 4
			else:
				action = 3
			
	return action

In [7]:
def basic_policy(observation):
  action_oh = [0,0,0,0,0]
  action = 0
  
  dist_re = observation[0]
  dist_de = observation[1]
  dist_le = observation[2]
  dist_ue = observation[3]
  dist_urd = observation[4]
  dist_rdd = observation[5]
  dist_dld = observation[6]
  dist_lud = observation[7]
	
  dist_t_re = observation[8]
  dist_t_de = observation[9]
  dist_t_le = observation[10]
  dist_t_ue = observation[11]
  dist_t_urd = observation[12]
  dist_t_rdd = observation[13]
  dist_t_dld = observation[14]
  dist_t_lud = observation[15]
  
  state = observation[16]
  
  botx,boty = observation[17],observation[18]
  #last_target_x,last_target_y = observation[19],observation[20]
  
  obstacle_distances = np.asarray([dist_re,dist_de,dist_le,dist_ue,dist_urd,dist_rdd,dist_dld,dist_lud])
  obstacle_distances_edge = np.asarray([dist_re,dist_de,dist_le,dist_ue])
  obstacle_distances_diagonal = np.asarray([dist_urd,dist_rdd,dist_dld,dist_lud])
	
  min_dist_target = min(abs(dist_t_re),abs(dist_t_de),abs(dist_t_le),abs(dist_t_ue),abs(dist_t_urd),abs(dist_t_lud),abs(dist_t_rdd),abs(dist_t_dld))
  min_dist_obstacle = np.min(obstacle_distances)
  collision_direction = np.argmin(obstacle_distances)
  run_direction = np.argmax(obstacle_distances_edge)
  
  """
  STATE INFORMATION:
  
  Finish State(FS)----->4
  Winning State(WS)---->3
  Safe State(SS)------->2
  Non-Safe State(NS)--->1
  Collision State(CS)-->0		
  """
  """
  ACTIONS:
  
  Nothing ----> 0
  Left -------> 1
  Right ------> 2
  Up ---------> 3
  Down -------> 4
  """
	 
  toss = random.randrange(0,1000)%2
  action = 0
  if state==3:
    if abs(dist_t_re)==min_dist_target:
      action = 2
    elif abs(dist_t_de) == min_dist_target:
      action = 4
    elif abs(dist_t_le) == min_dist_target:
      action = 1
    elif abs(dist_t_ue) == min_dist_target:
      action = 3
    elif abs(dist_t_urd) == min_dist_target:
      if toss==0:
        action = 3
      else:
        action = 2
    elif abs(dist_t_rdd) == min_dist_target:
      if toss==0:
        action = 2
      else:
        action = 4
    elif abs(dist_t_dld) == min_dist_target:
      if toss==0:
        action = 4
      else:
        action = 1
    elif abs(dist_t_lud) == min_dist_target:
      if toss==0:
        action = 1
      else:
        action = 3	
  elif state==2:
    if min_dist_target<5000:
      if abs(dist_t_re)==min_dist_target:
        action = 2
      elif abs(dist_t_de) == min_dist_target:
        action = 4
      elif abs(dist_t_le) == min_dist_target:
        action = 1
      elif abs(dist_t_ue) == min_dist_target:
        action = 3
      elif abs(dist_t_urd) == min_dist_target:
        if toss==0:
          action = 3
        else:
          action = 2
      elif abs(dist_t_rdd) == min_dist_target:
        if toss==0:
          action = 2
        else:
          action = 4
      elif abs(dist_t_dld) == min_dist_target:
        if toss==0:
          action = 4
        else:
          action = 1
      elif abs(dist_t_lud) == min_dist_target:
        if toss==0:
          action = 1
        else:
          action = 3
    else:
      #action = bot_velocity(last_target_x,last_target_y,botx,boty)
      action = random.randint(0,4)
  elif state==1:
    if run_direction==0:
      action = 2
    elif run_direction==1:
      action = 4
    elif run_direction==2:
      action = 1
    elif run_direction==3:
      action = 3
	
  action_oh[action] = 1
  
  return action_oh

In [8]:
def play4data(gen, epsilon):
    training_data = []
    for i in range(num_games):

        score = 0
        data = []
        prev_observation = []
        env.reset()
        done = False
        #d = deque()

        for _ in range(goal_steps):

            #env.render()
	
            if len(prev_observation)==0 or np.random.rand() <= epsilon:
                #theta = np.random.randn(possible_actions)
                if len(prev_observation)==0:
                  theta = np.random.rand(possible_actions)
                else:
                  theta = basic_policy(prev_observation)
            else:
                theta = q_model.predict(np.array(prev_observation).reshape(-1, 19))[0]
                #print(np.array(prev_observation).reshape(-1, 17).shape)

            action = np.argmax(theta)

            sensors, reward, done, info = env.step(action)

            score += reward
            
            observation = np.asarray([sensors['sensorRE'][0],
                						sensors['sensorDE'][0],
                						sensors['sensorLE'][0],
                						sensors['sensorUE'][0],
                						sensors['sensorURD'][0][0],
                						sensors['sensorRDD'][0][0],
                						sensors['sensorDLD'][0][0],
                						sensors['sensorLUD'][0][0],
                						sensors['sensorRE'][1],
                						sensors['sensorDE'][1],
                						sensors['sensorLE'][1],
                						sensors['sensorUE'][1],
                						sensors['sensorURD'][1][0],
                						sensors['sensorRDD'][1][0],
                						sensors['sensorDLD'][1][0],
                						sensors['sensorLUD'][1][0],
                						info['State'],
                            info['Bot'][0],
                            info['Bot'][1]])
                            #info['Last_Target'][0],
                            #info['Last_Target'][1]])	

            #if len(prev_observation)>0:
                #d.append(prev_observation)
            #if len(d) > 4:
                #d.popleft()
            
            if len(prev_observation) > 0:
                training_data.append([prev_observation, action, reward, done])

            prev_observation = observation

            if done:
                break

        print('gen {1} game {0}: '.format(i, gen) + str(score))

        
    env.reset()
    return training_data

In [9]:
"""# exact model described in DeepMind paper, just added a layer to end for 18 to 4
def simple_model(num_frames, width, height, lr, output=9, model_name='intelAI.model'):
    network = input_data(shape=[None, num_frames, width, height], name='input')
    conv1 = conv_2d(network, 8, 32,strides=4, activation='relu', name='conv1')
    conv2 = conv_2d(conv1, 4, 64, strides=2, activation='relu', name='conv2')
    conv3 = conv_2d(conv2, 3, 64, strides=1, activation='relu', name='conv3')
    fc4 = fully_connected(conv3, 512, activation='relu')
    # fc5 = fully_connected(fc4, 18, activation='relu')
    fc6 = fully_connected(fc4, output, activation='relu')
    
    # rmsprop = RMSProp(learning_rate=0.1, decay=0.99, momentum=0.0, epsilon=1e-6)
    network = regression(fc6, optimizer='adam',
                         loss='mean_square',
                         learning_rate=lr, batch_size = 32, name='targets')

    model = tflearn.DNN(network,
                        max_checkpoints=0, tensorboard_verbose=0, tensorboard_dir='log')
    return model"""

def simple_model():
    model = Sequential()
    model.add(Dense(128,input_dim = 19, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(128, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(5, kernel_initializer='normal', activation='relu'))
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

In [10]:
#graph1 = tf.Graph()
#with graph1.as_default():
target_model = simple_model()
#graph2 = tf.Graph()
#with graph2.as_default():
q_model = simple_model()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               2560      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
Total params: 19,717
Trainable params: 19,717
Non-trainable params: 0
_________________________________________________________________


In [11]:
"""def train2play(training_data):

    targets = np.zeros((len(training_data), possible_actions))
    state = np.array([i[0] for i in training_data]).reshape(-1,4,17)
    print(state.shape)
    #state = np.array([i[0] for i in training_data]).reshape(-1, 4, height, width)
    action = [i[1] for i in training_data]
    reward = np.asarray([i[2] for i in training_data])
    #reward = np.sign([i[2] for i in training_data])
    done = [i[3] for i in training_data]
    flag = 0
    
    for i in range(len(training_data)):
        #with graph2.as_default():
            #print(state[i-1 if i > 0 else 0].reshape(-1,4,17).shape)
        if flag==1:
          targets[i] = q_model.predict(state[i-1 if i > 0 else 0].reshape(-1,4,17))
            #print(state[i-1 if i > 0 else 0].shape)

        #with graph1.as_default():
          Q_sa = target_model.predict(state[i].reshape(-1,4,17))
            #print(state[i].shape)

        if done:
            targets[i, action[i]] = reward[i]
        else:
            targets[i, action[i]] = reward[i] + gamma * np.max(Q_sa)


    #with graph2.as_default():
    q_model.fit(state,targets,validation_split=0.1,epochs=5,batch_size=8,verbose=1)
    flag = 1
        #q_model.fit({'input': state}, {'targets': targets}, n_epoch=5, snapshot_step=500, show_metric=True, run_id='openai_learning')"""

"def train2play(training_data):\n\n    targets = np.zeros((len(training_data), possible_actions))\n    state = np.array([i[0] for i in training_data]).reshape(-1,4,17)\n    print(state.shape)\n    #state = np.array([i[0] for i in training_data]).reshape(-1, 4, height, width)\n    action = [i[1] for i in training_data]\n    reward = np.asarray([i[2] for i in training_data])\n    #reward = np.sign([i[2] for i in training_data])\n    done = [i[3] for i in training_data]\n    flag = 0\n    \n    for i in range(len(training_data)):\n        #with graph2.as_default():\n            #print(state[i-1 if i > 0 else 0].reshape(-1,4,17).shape)\n        if flag==1:\n          targets[i] = q_model.predict(state[i-1 if i > 0 else 0].reshape(-1,4,17))\n            #print(state[i-1 if i > 0 else 0].shape)\n\n        #with graph1.as_default():\n          Q_sa = target_model.predict(state[i].reshape(-1,4,17))\n            #print(state[i].shape)\n\n        if done:\n            targets[i, action[i]] = rew

In [12]:
def train2play(training_data):
  
  
  targets = np.zeros((len(training_data), possible_actions))
  state = np.array([i[0] for i in training_data])
  print(state.shape)
  action = np.array([i[1] for i in training_data])
  reward = np.array([i[2] for i in training_data])
  done = np.array([i[3] for i in training_data])
  
  for i in range(len(training_data)):
    #with graph2.as_default():
    result = q_model.predict(state[i].reshape(-1,19))[0]
    #print(result)
    targets[i] = result

    #with graph1.as_default():
    Q_sa = target_model.predict(state[i].reshape(-1,19))[0]
   
    if done[i]:
      targets[i, action[i]] = reward[i]
    else:
      targets[i, action[i]] = reward[i] + gamma * np.max(Q_sa)
      
  q_model.fit(state,targets,validation_split=0.15,epochs=20,batch_size=128,verbose=1)
  

In [13]:
for gen in range(generations):
    
    epsilon = get_epsilon(epsilon, gen)
    
    training_data =  play4data(gen, epsilon)
    train2play(training_data)

    #with graph2.as_default():
    q_model.save(MODEL_NAME.format(gen)+'.h5')

    if gen % 10 == 0:
        #with graph1.as_default():
        target_model = load_model(MODEL_NAME.format(gen)+'.h5')

1.0
gen 0 game 0: -7
gen 0 game 1: -25
gen 0 game 2: -25
gen 0 game 3: -9
gen 0 game 4: 89
gen 0 game 5: -9
gen 0 game 6: 100
gen 0 game 7: -9
gen 0 game 8: -1
gen 0 game 9: 82
gen 0 game 10: -1
gen 0 game 11: -104
gen 0 game 12: -25
gen 0 game 13: 102
gen 0 game 14: -28
gen 0 game 15: -100
gen 0 game 16: 66
gen 0 game 17: -10
gen 0 game 18: -15
gen 0 game 19: -6
gen 0 game 20: -7
gen 0 game 21: -21
gen 0 game 22: 100
gen 0 game 23: -42
gen 0 game 24: -39
gen 0 game 25: -26
gen 0 game 26: -5
gen 0 game 27: 103
gen 0 game 28: -42
gen 0 game 29: -27
gen 0 game 30: -7
gen 0 game 31: 98
gen 0 game 32: -12
gen 0 game 33: 99
gen 0 game 34: 104
gen 0 game 35: 103
gen 0 game 36: -100
gen 0 game 37: 99
gen 0 game 38: 102
gen 0 game 39: -2
gen 0 game 40: 103
gen 0 game 41: 82
gen 0 game 42: -12
gen 0 game 43: -20
gen 0 game 44: 90
gen 0 game 45: 99
gen 0 game 46: 74
gen 0 game 47: 101
gen 0 game 48: 96
gen 0 game 49: 97
(33783, 19)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Train on 28715 samples, validate on 5068 samples
Epoch 1/20
28715/28715 [==============================] - 1s 23us/sample - loss: 4449.8583 - acc: 0.9517 - val_loss: 3627.5605 - val_acc: 0.9568
Epoch 2/20
28715/28715 [==============================] - 0s 16us/sample - loss: 3827.4135 - acc: 0.9643 - val_loss: 3579.5567 - val_acc: 0.9556
Epoch 3/20
28715/28715 [==============================] - 0s 16us/sample - loss: 3779.0669 - acc: 0.9639 - val_loss: 3589.0216 - val_acc: 0.9528
Epoch 4/20
28715/28715 [==============================] - 0s 16us/sample - loss: 3733.4884 - acc: 0.9643 - val_loss: 3580.4912 - val_acc: 0.9528
Epoch 5/20
28715/28715 [==============================] - 0s 15us/sample - loss: 3724.5781 - acc: 0.9640 - val_loss: 3603.7606 - val_acc: 0.9566
Epoch 6/20
28715/28715 [==============================] - 0s 16us/sample - loss: 3709.8186 - acc: 0.9648 - val_loss: 3523.8003 - val_acc: 0.9625
Epoch 7/20
28715/28715 [==============================] - 0s 16us/sample - loss: 

In [14]:
q_model.save(MODEL_NAME.format(game)+'_final.h5')